import numpy as np
import scipy as sp

In [53]:
import numpy as np
import scipy as sp

In [54]:
from quadrotor.dynamics import QuadrotorDynamicsBase, QuadrotorState
from quadrotor.controller import ControllerBase, QuadrotorCommands
from quadrotor.trajectory import TrajectoryBase, TrajectoryState

In [55]:
from sym import Rot3

def dRot3(R: Rot3, omega: np.ndarray) -> Rot3:
    """
    NOTE(roy): If interested, we can discuss the math here later. There are other
    ways of doing this but to keep the ode_int formulation, we specifically require
    dq/dt. Refer to 'Baseile Graf - Quaternions and Dynamics' or (for intuition)
    https://math.stackexchange.com/questions/1896379/how-to-use-the-quaternion-derivative
    """
    quat = np.array(R.to_storage())  # Convert to raw quaternion
    (q0, q1, q2, q3) = quat  # xyzw storage, see https://symforce.org/api-gen-cpp/class/classsym_1_1Rot3.html

    G = np.array([[ q3,  q2, -q1, -q0],
                [-q2,  q3,  q0, -q1],
                [ q1, -q0,  q3, -q2]])
    quat_dot = (G.T @ omega) / 2
    return Rot3.from_storage(quat_dot)

In [56]:
from dataclasses import dataclass, field

@dataclass
class Params:
    """A little utility class to hold our quadrotor parameters"""
    mass: float = 1.352  # [kg]

    # NOTE(roy) this is a matrix now as discussed!
    # NOTE(roy) don't get scared by the field/default_factory thing! It's just a way to do default field initialization for dataclasses,
    # see https://docs.python.org/3/library/dataclasses.html. Don't worry about it!
    inertia: np.ndarray = field(default_factory=lambda: np.array([
        [9.8e-3, 0, 0],
        [0, 10.02e-3, 0],
        [0, 0, 18.6e-3],
    ]))

    inertia_inverse: np.ndarray = field(default_factory=lambda: np.linalg.inv(
        np.array([
        [9.8e-3, 0, 0],
        [0, 10.02e-3, 0],
        [0, 0, 18.6e-3],
        ])))

    rotor_diameter: float = 10 * 0.0254  # [m] 10 inches

    static_thrust_coefficient: float = 0.14553  # [-] Thrust coefficient
    static_torque_coefficient: float = 0.01047  # [-] Torque coefficient

    # NOTE(roy) assume this is the same front-to-back and left-to-right
    arm_length: float = 0.3814 / 2.0  # [m] Arm length

    # Universal constants
    g: float = 9.80665  # [m/s2] Gravity
    rho: float = 1.225  # [kg/m3] Sea-level density

    def rotor_model(self, static_coefficient: float) -> float:
        # NOTE(roy) I'm just including this for completeness sake, this means we can e.g. see how air density
        # influences our simulation. Refer to this source for derivation:
        # https://web.mit.edu/16.unified/www/FALL/thermodynamics/notes/node86.html
        return self.rho * static_coefficient * (self.rotor_diameter ** 4) / (4 * np.pi ** 2)

    # NOTE(roy) if you're unsure what @property decorators do: they make it so that we can call k_thrust as if it was just a class
    # variable. I.e. we'd write params.k_thrust now, rather than params.k_thrust()
    @property
    def k_thrust(self):
        return self.rotor_model(self.static_thrust_coefficient)

    @property
    def k_torque(self):
        return self.rotor_model(self.static_torque_coefficient)

# Simply treat as a global
p = Params()
mixing_matrix = np.array([
            [1, 1, 1, 1],
            [0, p.arm_length, 0, -p.arm_length],
            [-p.arm_length, 0, p.arm_length, 0],
            [p.k_torque / p.k_thrust, -p.k_torque / p.k_thrust, p.k_torque / p.k_thrust, -p.k_torque / p.k_thrust]
        ])

### Writing a controller for our drone
In this notebook, we'll start writing a feedback controller for our model! Again, I will be leaving you guys alone a little bit more.

#### Recommended steps
- Copy your 3D quadrotor model from the previous lecture
- Work your way down in the `Controller` class
    - I implemented a basic structure, but feel free to diverge from this if you like!
    - Write the inverse mixing matrix formulation to from **u** to rotor rates
    - As discussed, the first week I expect you to just set up control in *z* to warm up!

### The dynamics

In [57]:
# NOTE(roy) Copy your 3D Quadrotor here!
# NOTE(roy) please copy your previous assignment as a starting point!

class FullQuadrotorDynamics(QuadrotorDynamicsBase):
    def __init__(self) -> None:
        super().__init__()

    @staticmethod
    def rotor_thrust_model(rotor_rates: np.ndarray) -> np.ndarray:
        # NOTE(roy): Implement the simple rotor trust model we discussed in the slides
        # (For fun) to see how complex this can get, see for example: https://arxiv.org/pdf/1601.00733.pdf
        return p.k_thrust * rotor_rates ** 2

   #Implement the mixing matrix (i.e. compute u1 and u2 from the rotor rates)
    
    
    def step(self, t: float, input: QuadrotorCommands) -> QuadrotorState:
        F1, F2, F3, F4 = FullQuadrotorDynamics.rotor_thrust_model(input.rotor_rates) 

        # NOTE(roy) Implement the thrust/torque calculation we discussed in the slides
        thrust_force_vector = np.array([F1, F2, F3, F4])
        u_matrix = mixing_matrix @ thrust_force_vector
        u1 = u_matrix[0]
        u2 = u_matrix[1:] 

        # NOTE(roy) this local function is a wrapper around the class state_derivative function
        # all this does is convert the our state dataclass from/into a vector that the scipy
        # integration solver can work with. See dynamics.py for the definition of these
        # from/to state vector functions.
        def state_derivative_local_wrapped(t: float, state: np.ndarray) -> np.ndarray:
            return FullQuadrotorDynamics.state_derivative(
                QuadrotorState.from_state_vector(state),
                u1,
                u2,
            ).to_state_vector()

        state_vector = self.state.to_state_vector()
        #raise NotImplementedError("Use the scipy function solve_ivp to integrate to the next timestep")
        

        # NOTE(roy) solution = sp.integrate.solve_ivp(....
        solution = sp.integrate.solve_ivp(
            fun=state_derivative_local_wrapped,
            t_span=[0, self.dt],
            y0=state_vector,
            method='RK45'
        )

        self.state = QuadrotorState.from_state_vector(solution['y'][:, -1])
        return self.state

    #Modify the state_derivative functions to take a vector for u2 rather than a float (because now of course that's 3 numbers!)
    #Modify the accel formulation in your model to 3D (see equations in slides)
    #Modify the angular_accel formulation in your model to 3D (see equations in slides)
    @staticmethod
    def state_derivative(state: QuadrotorState, u1: float, u2: np.ndarray) -> QuadrotorState:
        rotation_body_to_world = state.orientation.to_rotation_matrix()
        gravity_vector = np.array([0, 0, -p.mass * p.g])
        u1_vector = np.array([0, 0, u1])
        accel = (gravity_vector + np.matmul(rotation_body_to_world, u1_vector)) / p.mass
        angular_accel = np.matmul(p.inertia_inverse, 
                                  (u2 - np.cross(state.angular_velocity, 
                                                                    np.matmul(p.inertia, 
                                                                              state.angular_velocity))))
        
        # NOTE(roy) this might look a little confusing at first glance, but we're populating DERIVATIVES
        # of the state here! So position -> velocity, velocity -> acceleration, etc...
        # If you can think of a more readable way to do this (e.g. defining a QuadrotorStateDerivative class)
        # feel free to open a pull-request into the upstream repository!
        state_derivative = QuadrotorState(
            position=state.velocity,
            velocity=accel,
            orientation=dRot3(state.orientation, state.angular_velocity),
            angular_velocity=angular_accel,
        )

        return state_derivative



### The controller

In [58]:
@dataclass
class ControllerParams:
    """The same as above but for the controller gains"""

    # NOTE(roy) diag just means we get a matrix with these values on the diagonal,
    # so these are 3x3 matrices (print them if you're unsure!)
    # We use this formulation because it makes the math a bit more readable later on!
    K_p: np.ndarray = field(default_factory=lambda: np.diag([0, 0, 100]))
    K_d: np.ndarray = field(default_factory=lambda: np.diag([0, 0, 20]))

    # Min/max rotor rates!
    rotor_rate_min: float = 180
    rotor_rate_max: float = 600

controller_p = ControllerParams()

In [59]:
class Controller(ControllerBase):
    @staticmethod
    def rotor_rates_from_u(u: np.ndarray) -> np.ndarray:
        # NOTE(roy) implement your conversion from u (so that's [u1, u2_1, u2_2, u2_3] like we discussed, a 4-sized vector)
        # to rotor rates (which is also a four-sized vector)

        # NOTE(roy) the steps here are:
        #  1. convert u -> forces with the inverse mixing matrix
        #  2. convert forces -> rotor rates by inverting the rotor rate model
        #  3. limit the rotor rates with rotor_rate_min/rotor_rate_max above!
        #raise NotImplementedError("rotor_rates_from_u not implemented!")

        forces = np.linalg.inv(mixing_matrix) @ u
        # print('forces = ', forces)
        forces = np.maximum(forces, 0) # replaces negative forces with 0 for squareroot below
        rotor_rates = np.sqrt(forces / p.k_thrust)
        for i in np.arange(len(rotor_rates)): # forces below min and above max changed to min and max rotor rates
            if rotor_rates[i] > controller_p.rotor_rate_max:
                rotor_rates[i] = controller_p.rotor_rate_max
            if rotor_rates[i] < controller_p.rotor_rate_min:
                rotor_rates[i] = controller_p.rotor_rate_min
        # print('rotor_rates = ', rotor_rates)
        return rotor_rates

    def step(self, t: float, trajectory: TrajectoryState, state: QuadrotorState) -> QuadrotorCommands:
        desired_position, desired_velocity = trajectory.position, trajectory.velocity
        actual_position, actual_velocity = state.position, state.velocity
        
        # NOTE(roy) first compute the desired acceleration using the equation from the lectures!
        # I already extracted the values you need to compute it above
        state_error = desired_position - actual_position
        state_error_derivative = desired_velocity - actual_velocity
        commanded_acceleration = controller_p.K_d @ state_error + controller_p.K_p @ state_error_derivative  # Your equation here!

        # NOTE(roy) now compute u_1 from the commanded acceleration using the linearized equation from the lectures!
        u_1 = p.mass * (p.g + commanded_acceleration[2])  # Your equation here!

        # NOTE(roy) we're just doing thrust this week, set u_2 to just be zeros!
        u_2 = np.zeros(3)
        u = np.array([u_1, *u_2])

        rotor_rates = Controller.rotor_rates_from_u(u)
        return QuadrotorCommands(rotor_rates)

### The trajectory
Because we're now finally doing closed-loop control, we also need to set desired states in the trajectory!
I've set up something simple, please mess around with it!

In [60]:
class JumpTrajectory(TrajectoryBase):
    def eval(self, t: float):
        altitude = 0 if t < 1.0 else 1.0
        position = np.array([0, 0, altitude])

        return TrajectoryState(t, position=position)

### Running it!

In [61]:
from quadrotor.simulator import SimulatorBase, SimulatorState

# This might be slow to run the first time!
from quadrotor.renderer import animate_k3d

In [62]:
def run_and_render(initial_state: QuadrotorState, t_total: int = 2.0):
    sim = SimulatorBase(
        dt=0.01,
        dynamics=FullQuadrotorDynamics(),
        controller=Controller(),
        trajectory=JumpTrajectory(),
        initial_state=initial_state,
        t_total=t_total,
    )
    
    # Run the simulator
    output = sim.simulate()

    # K3D renderer
    plot = animate_k3d(output)
    
    plot.display()
    plot.start_auto_play()

In [63]:
# NOTE(roy) play with this variable and see what happens!
initial_state = QuadrotorState(
    position=np.zeros(3),
    velocity=np.array([0, 0, 0]),
    orientation=Rot3.from_yaw_pitch_roll(0, 0, 0),
    angular_velocity=np.zeros(3),
)

run_and_render(initial_state, 5.0)

forces =  [ 4.41953027 16.00718173  4.41953027 -7.16812119]
rotor_rates =  [484.90446412 600.         484.90446412 180.        ]
forces =  [ 4.90797221 17.77627899  4.90797221 -7.96033457]
rotor_rates =  [510.9979683 600.        510.9979683 180.       ]
forces =  [ 5.23796376 18.97148176  5.23796376 -8.49555423]
rotor_rates =  [527.89721507 600.         527.89721507 180.        ]
forces =  [ 5.48233136 19.85656146  5.48233136 -8.89189874]
rotor_rates =  [540.07088689 600.         540.07088689 180.        ]
forces =  [ 5.72881016 20.74928777  5.72881016 -9.29166745]
rotor_rates =  [552.07787344 600.         552.07787344 180.        ]
forces =  [ 6.10248646 22.10271315  6.10248646 -9.89774023]
rotor_rates =  [569.79881056 600.         569.79881056 180.        ]
forces =  [  6.78881012  24.58852203   6.78881012 -11.01090179]
rotor_rates =  [600. 600. 600. 180.]
forces =  [  8.07682421  29.25360505   8.07682421 -13.09995663]
rotor_rates =  [600. 600. 600. 180.]
forces =  [ 10.58936528  38.

Output()

### Looking at the controller behavior
If all is well, the quadrotor should have a stable hover and after 1 second jump up 1 meter (with the `JumpTrajectory`)

Try some experiments:
- What happens if you increase/descrease `K_p`?
- What happens if you increase/decrease `K_d`?
- What happens if you use a slightly different mass in the controller (so the controller and dynamics have a different estimate for the quadrotor mass)? How do you think you could solve the behavior you see?

### Commit your updated notebook to your fork!